# Data loading 
Next cells are used to load the data we need.

In [1]:
import os

os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1

import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [7]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

URM_train_ICM_all_IALS_group_0 = sps.vstack([URM_all*(1-0.5299554043024671), ICM_all.T*0.5299554043024671])
recommender_IALS_group_0 = IALSRecommender_implicit(URM_train_ICM_all_IALS_group_0)
recommender_IALS_group_0.fit(n_factors = 864, regularization = 0.7981462652421099, iterations=100)

Recommender_Base_Class: URM Detected 2 (0.01 %) cold users.


In [8]:
URM_train_ICM_all_IALS_group_1 = sps.vstack([URM_all*(1-0.6790082920718808), ICM_all.T*0.6790082920718808])
recommender_IALS_group_1 = IALSRecommender_implicit(URM_train_ICM_all_IALS_group_1)
recommender_IALS_group_1.fit(n_factors = 736, regularization =0.4021244259692904, iterations=77)

Recommender_Base_Class: URM Detected 2 (0.01 %) cold users.


In [9]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

URM_train_ICM_all_RP3 = sps.vstack([URM_all, ICM_all.T])
recommender_rp3Beta_group_0 = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta_group_0.fit(topK=196, alpha = 0.35248383404238176, beta=0.14410506792619115, implicit = False) 

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.


In [10]:
URM_train_ICM_all_RP3 = sps.vstack([URM_all, ICM_all.T])
recommender_rp3Beta_group_1 = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta_group_1.fit(topK=396, alpha = 0.41137245632718406, beta=0.29679690096781264, implicit = False) 

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.


In [11]:
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1-0.421), ICM_all.T*0.421])
recommender_SLIM = SLIMElasticNetRecommender(URM_train_ICM_all_SLIM)
recommender_SLIM.fit(topK=996, alpha = 0.000182, l1_ratio = 4.459e-06)
recommender_SLIM.W_sparse.data = np.power(recommender_SLIM.W_sparse.data, 0.8546)

SLIMElasticNetRecommender: URM Detected 2 (0.01 %) cold users.


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0021116770803928375, tolerance: 0.0008219736628234386
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0009993426501750946, tolerance: 0.0005707712844014168
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0002247089723823592, tolerance: 0.00011493572674226016
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\en

SLIMElasticNetRecommender: Processed 4391 ( 16.90% ) in 5.00 min. Items per second: 14.63


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0030994615517556667, tolerance: 0.0006375000230036676
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.050417460501194, tolerance: 0.003089803270995617
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05425300449132919, tolerance: 0.0028581060469150543
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_en

SLIMElasticNetRecommender: Processed 8780 ( 33.80% ) in 10.00 min. Items per second: 14.63


...

SLIMElasticNetRecommender: Processed 17496 ( 67.36% ) in 20.00 min. Items per second: 14.58


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.004778327886015177, tolerance: 0.001754127093590796
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.007375245448201895, tolerance: 0.0023492868058383465
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010946077294647694, tolerance: 0.002137906616553664
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_

SLIMElasticNetRecommender: Processed 21860 ( 84.16% ) in 25.01 min. Items per second: 14.57


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001274635549634695, tolerance: 0.0009633576846681535
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00564293609932065, tolerance: 0.0012134457938373089
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.004195852670818567, tolerance: 0.0015368710737675428
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my

SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 29.76 min. Items per second: 14.54


In [12]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1- 0.6229636472829045), ICM_all.T* 0.6229636472829045])
recommender_SLIM_BPR = SLIM_BPR_Cython(URM_train_ICM_all_SLIM, recompile_cython=False, verbose=False)
recommender_SLIM_BPR.fit(epochs=87, sgd_mode= "adam", learning_rate= 6.12252603066224e-06, topK = 805, batch_size = 35, lambda_i=3.0399585540912118e-05, lambda_j=0.00012176512055401498 )

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 87. Elapsed time 0.08 sec
SLIM_BPR_Recommender: Epoch 2 of 87. Elapsed time 0.17 sec
SLIM_BPR_Recommender: Epoch 3 of 87. Elapsed time 0.25 sec
SLIM_BPR_Recommender: Epoch 4 of 87. Elapsed time 0.34 sec
SLIM_BPR_Recommender: Epoch 5 of 87. Elapsed time 0.42 sec
SLIM_BPR_Recommender: Epoch 6 of 87. Elapsed time 0.50 sec
SLIM_BPR_Recommender: Epoch 7 of 87. Elapsed time 0.58 sec
SLIM_BPR_Recommender: Epoch 8 of 87. Elapsed time 0.67 sec
SLIM_BPR_Recommender: Epoch 9 of 87. Elapsed time 0.75 sec
SLIM_BPR_Recommender: Epoch 10 of 87. Elapsed time 0.84 sec
SLIM_BPR_Recommender: Epoch 11 of 87. Elapsed time 0.92 sec
SLIM_BPR_Recommender: Epoch 12 of 87. Elapsed time 1.01 sec
SLIM_BPR_Recommender: Epoch 13 of 87. Elapsed time 1.09 sec
SLIM_BPR_Recommender: Epoch 14 of 87. Elapsed time 1.18 sec
SLIM_BPR_Recommender: Epoch 15 of 87. Elapsed time 1.26 sec
SLIM_BPR_Recommender: Epoch 16 of 87. Elapsed time 1.34

In [13]:
grouped_users = dataset.groupby(['row']).count()
sorted_users = grouped_users.sort_values(by=['col'], ascending=True)
sorted_users = sorted_users.index.to_numpy()
print(sorted_users)
block_size = int(len(sorted_users)*0.75)

start_pos_group_0 = 0*block_size
end_pos_group_0 = min((0+1)*block_size, len(sorted_users))
users_in_group_0 = sorted_users[start_pos_group_0:end_pos_group_0]
users_not_in_group_0_flag = np.isin(sorted_users, users_in_group_0, invert = True)
users_not_in_group_0 = sorted_users[users_not_in_group_0_flag]
print(users_not_in_group_0)

start_pos_group_1 = 1*block_size
end_pos_group_1 = min((1+1)*block_size, len(sorted_users))
users_in_group_1 = sorted_users[start_pos_group_1:end_pos_group_1]
users_not_in_group_1_flag = np.isin(sorted_users, users_in_group_1, invert = True)
users_not_in_group_1 = sorted_users[users_not_in_group_1_flag]
print(users_not_in_group_1)

[6012 3608 3171 ... 1863 2282 7382]
[6468 7223 1795 ... 1863 2282 7382]
[6012 3608 3171 ... 6212 2053 7222]


In [14]:
test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [15]:
user_id = test_users['user_id'].to_numpy()
users_in_group_0 = users_in_group_0[np.isin(users_in_group_0, user_id)]
print(len(users_in_group_0))


users_middle_flag = np.isin(user_id, users_in_group_0, invert = True)
users_in_group_1 = user_id[users_middle_flag]
print(len(users_in_group_1))



5957
1987


In [16]:
#group 0 recommender 
#{'alpha': 0.21888048647913086, 'beta': 0.7404146260887783, 'gamma': 0.08659805835601905, 'phi': 0.25025824053338225}
from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender

URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])
hybrid_0 = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS_group_0,recommender_rp3Beta_group_0,recommender_SLIM,recommender_SLIM_BPR], 4)
hybrid_0.fit([0.21888048647913086,  0.7404146260887783, 0.08659805835601905,  0.25025824053338225])
#evaluator_validation.evaluateRecommender(hybrid)

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 4


In [17]:
#group 1 recommender
#{'alpha': 0.1040533892168851, 'beta': 0.52014749085858, 'gamma': 0.12490000806750567, 'phi': 0.6835008850054951}

URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])
hybrid_1 = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS_group_1,recommender_rp3Beta_group_1,recommender_SLIM,recommender_SLIM_BPR], 4)
hybrid_1.fit([0.1040533892168851,  0.52014749085858, 0.12490000806750567,  0.6835008850054951])

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 4


In [18]:
recommendations_first = hybrid_0.recommend(users_in_group_0,cutoff = 10)
recommendations_last = hybrid_1.recommend(users_in_group_1,cutoff = 10)

In [19]:
for index in range(len(recommendations_first)):
    recommendations_first[index]=np.array(recommendations_first[index])

print(len(recommendations_first))

for index in range(len(recommendations_last)):
    recommendations_last[index]=np.array(recommendations_last[index])

print(len(recommendations_last))

5957
1987


In [20]:
elementi = test_users['user_id'].to_numpy()
recommendations = []

for index in elementi:
    if index in users_in_group_0:
        local_index, = np.where(users_in_group_0 == index) 
        recommendations.append(recommendations_first[local_index[0]])
    
    if index in users_in_group_1:
        local_index, = np.where(users_in_group_1 == index) 
        recommendations.append(recommendations_last[local_index[0]])
    
    print(index, " - ", local_index[0])

print(recommendations)

0  -  275
1  -  4074
2  -  963
3  -  1024
4  -  4549
5  -  0
6  -  2690
7  -  1
8  -  1918
9  -  4101
10  -  4104
11  -  2
12  -  1984
13  -  985
14  -  2203
15  -  3
16  -  1052
17  -  4
18  -  5898
19  -  1070
20  -  4485
21  -  5
22  -  6
23  -  4788
24  -  1415
25  -  4075
26  -  3549
27  -  4855
28  -  4317
29  -  5639
30  -  4834
31  -  1181
32  -  1080
33  -  2407
34  -  1768
35  -  1765
36  -  7
37  -  8
38  -  4450
39  -  1013
40  -  4329
41  -  5911
42  -  5704
43  -  9
44  -  1640
45  -  2753
46  -  10
47  -  4657
48  -  11
49  -  2774
50  -  3979
51  -  12
52  -  3912
53  -  5122
54  -  1044
55  -  1023
56  -  2483
57  -  13
58  -  971
59  -  1836
60  -  3949
61  -  4428
62  -  14
63  -  4999
64  -  4973
65  -  3436
66  -  5311
67  -  965
68  -  15
69  -  1007
70  -  1035
71  -  1395
72  -  16
73  -  17
74  -  18
75  -  19
76  -  20
77  -  21
78  -  2734
79  -  2009
80  -  1913
81  -  22
82  -  23
83  -  3921
84  -  24
85  -  1081
86  -  1083
87  -  25
88  -  26
89  -  2079

In [21]:
'''user_id = test_users['user_id']
recommendations = hybrid.recommend(user_id,cutoff = 10)'''


"user_id = test_users['user_id']\nrecommendations = hybrid.recommend(user_id,cutoff = 10)"

In [22]:
test_users['item_list']= recommendations

test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

test_users
test_users.to_csv('submission.csv', index=False)
